# 04. 인터랙티브 시각화 대시보드

Plotly를 활용한 인터랙티브 분석 시각화

- UMAP 인터랙티브 산점도
- 맵별 승률 비교 막대 차트
- 피처 분포 박스플롯
- 개인 진단 도구 (accountId 입력)


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

BASE_DIR   = r'C:\배그분석'
OUTPUT_DIR = os.path.join(BASE_DIR, 'analysis_output')
MODEL_DIR  = os.path.join(OUTPUT_DIR, 'models')

with open(os.path.join(MODEL_DIR, 'erangel_model.pkl'), 'rb') as f:
    model = pickle.load(f)

FEATURE_COLS   = model['feature_cols']
cluster_names  = model['cluster_names']
PERSONA_LABELS = model['persona_labels']

PERSONA_COLORS = {
    '중앙 점령형':      '#E74C3C',
    '외곽 운영형':      '#2ECC71',
    '하이리스크 어태커': '#E67E22',
    '게릴라 운영형':    '#9B59B6',
    '노이즈':           '#95A5A6',
}

print('설정 완료')


In [ ]:
# 데이터 로드
MAPS = ['Erangel', 'Miramar', 'Taego', 'Rondo']
map_dfs = {}

for map_name in MAPS:
    path = os.path.join(OUTPUT_DIR, f'{map_name.lower()}_clustered.parquet')
    if os.path.exists(path):
        df_map = pd.read_parquet(path)
        df_map['map'] = map_name
        map_dfs[map_name] = df_map
        print(f'{map_name}: {len(df_map):,}명 로드')
    else:
        print(f'{map_name}: 파일 없음')

df_all = pd.concat(map_dfs.values(), ignore_index=True) if map_dfs else pd.DataFrame()
print(f'전체: {len(df_all):,}명')


In [ ]:
# 차트 1: UMAP 인터랙티브 산점도
df_erangel = map_dfs.get('Erangel', pd.DataFrame())

if len(df_erangel) > 0 and 'umap_x' in df_erangel.columns:
    df_plot = df_erangel[df_erangel['cluster'] >= 0].copy()

    hover_parts = [
        'persona',
        'drop_distance_from_path',
        'vehicle_use_ratio',
        'bluezone_exposure_ratio',
        'win_flag',
    ]
    hover_cols = [c for c in hover_parts if c in df_plot.columns]

    fig = px.scatter(
        df_plot,
        x='umap_x',
        y='umap_y',
        color='persona',
        hover_data=hover_cols,
        opacity=0.55,
        title='에란겔 전략 클러스터 - UMAP 분포',
        labels={'umap_x': 'UMAP 1', 'umap_y': 'UMAP 2', 'persona': '전략 유형'},
        width=950,
        height=620,
    )
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(title_font_size=16, legend_title='전략 유형')
    fig.show()
    fig.write_html(os.path.join(OUTPUT_DIR, 'umap_interactive.html'))
    print('저장: umap_interactive.html')
else:
    print('에란겔 데이터 없음')


In [ ]:
# 차트 2: 맵별 x 페르소나별 승률 비교
if len(df_all) > 0 and 'win_flag' in df_all.columns:
    summary = (
        df_all[df_all['cluster'] >= 0]
        .groupby(['map', 'persona'])
        .agg(
            n=('win_flag', 'count'),
            win_rate=('win_flag', 'mean'),
            top10_rate=('top10_flag', 'mean'),
        )
        .reset_index()
    )

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['맵별 x 페르소나별 우승률', '맵별 x 페르소나별 Top10 비율'],
    )

    personas = summary['persona'].unique()
    palette  = px.colors.qualitative.Set2

    for i, persona in enumerate(personas):
        sub   = summary[summary['persona'] == persona]
        color = palette[i % len(palette)]

        fig.add_trace(go.Bar(
            x=sub['map'], y=sub['win_rate'],
            name=persona, marker_color=color,
            legendgroup=persona,
            text=[f'{v:.1%}' for v in sub['win_rate']],
            textposition='auto',
        ), row=1, col=1)

        fig.add_trace(go.Bar(
            x=sub['map'], y=sub['top10_rate'],
            name=persona, marker_color=color,
            legendgroup=persona, showlegend=False,
            text=[f'{v:.1%}' for v in sub['top10_rate']],
            textposition='auto',
        ), row=1, col=2)

    fig.update_layout(
        title='전략 페르소나별 맵 성과 비교',
        title_font_size=16,
        barmode='group',
        height=520,
        width=1100,
        legend_title='전략 유형',
    )
    fig.show()
    fig.write_html(os.path.join(OUTPUT_DIR, 'map_winrate_comparison.html'))
    print('저장: map_winrate_comparison.html')
else:
    print('데이터 없음')


In [ ]:
# 차트 3: 피처 박스플롯
if len(df_erangel) > 0:
    df_box = df_erangel[df_erangel['cluster'] >= 0].copy()

    feature_display = {
        'drop_distance_from_path': '낙하 거리 (m)',
        'vehicle_use_ratio':        '차량 이용률',
        'bluezone_exposure_ratio':  '블루존 노출률',
        'rotation_timing_score':    '로테이션 타이밍',
        'safezone_edge_ratio':      '외곽 포지셔닝 비율',
        'altitude_variance':        '고도 변화량',
    }
    available = {k: v for k, v in feature_display.items() if k in df_box.columns}

    n_cols = 3
    n_rows = (len(available) + n_cols - 1) // n_cols

    fig = make_subplots(rows=n_rows, cols=n_cols,
                        subplot_titles=list(available.values()))

    palette = px.colors.qualitative.Set2
    personas = df_box['persona'].dropna().unique()

    for idx, (feat, feat_name) in enumerate(available.items()):
        row = idx // n_cols + 1
        col = idx % n_cols + 1
        for j, persona in enumerate(personas):
            sub = df_box[df_box['persona'] == persona][feat].dropna()
            fig.add_trace(go.Box(
                y=sub,
                name=persona,
                marker_color=palette[j % len(palette)],
                legendgroup=persona,
                showlegend=(idx == 0),
                boxmean=True,
            ), row=row, col=col)

    fig.update_layout(
        title='클러스터별 피처 분포',
        title_font_size=16,
        height=700,
        width=1100,
        legend_title='전략 유형',
    )
    fig.show()
    fig.write_html(os.path.join(OUTPUT_DIR, 'feature_boxplot.html'))
    print('저장: feature_boxplot.html')


In [ ]:
# 차트 4: 페르소나별 성과 요약 테이블
if len(df_all) > 0 and 'win_flag' in df_all.columns:
    perf = (
        df_all[df_all['cluster'] >= 0]
        .groupby('persona')
        .agg(
            플레이어수=('win_flag', 'count'),
            우승률=('win_flag', 'mean'),
            Top10_비율=('top10_flag', 'mean'),
            평균_킬=('kills', 'mean'),
            평균_데미지=('damageDealt', 'mean'),
            평균_순위=('winPlace', 'mean'),
        )
        .round(3)
        .reset_index()
    )
    perf['우승률']    = perf['우승률'].map('{:.1%}'.format)
    perf['Top10_비율'] = perf['Top10_비율'].map('{:.1%}'.format)
    perf['평균_킬']   = perf['평균_킬'].round(2)
    perf['평균_데미지'] = perf['평균_데미지'].round(0).astype(int)

    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(perf.columns),
            fill_color='#2C3E50',
            align='center',
            font=dict(color='white', size=12),
        ),
        cells=dict(
            values=[perf[col] for col in perf.columns],
            fill_color=[['#ECF0F1' if i % 2 == 0 else 'white'
                         for i in range(len(perf))]]
                        * len(perf.columns),
            align='center',
            font=dict(size=11),
        ),
    )])
    fig.update_layout(
        title='전략 페르소나별 종합 성과 요약',
        title_font_size=16,
        height=320,
        width=1000,
    )
    fig.show()
    fig.write_html(os.path.join(OUTPUT_DIR, 'persona_summary_table.html'))
    print('저장: persona_summary_table.html')


In [ ]:
# 개인 진단 도구
def diagnose_player(account_id, target_map='Erangel'):
    df_target = map_dfs.get(target_map, pd.DataFrame())
    if len(df_target) == 0:
        print(f'{target_map} 데이터 없음')
        return

    rows = df_target[df_target['accountId'] == account_id]
    if len(rows) == 0:
        print(f'플레이어 {account_id} 를 {target_map} 에서 찾을 수 없습니다.')
        return

    p = rows.iloc[0]
    persona = p.get('persona', '알 수 없음')

    print('=' * 55)
    print('  플레이어 전략 진단 리포트')
    print('=' * 55)
    print(f'  맵       : {target_map}')
    print(f'  AccountId: {account_id}')
    print(f'  페르소나  : {persona}')
    print('-' * 55)
    print('  [전략 지표]')

    metrics = [
        ('낙하 거리',        'drop_distance_from_path',  '{:.0f}cm'),
        ('초기 적 밀도',     'early_enemy_density',      '{:.1f}명'),
        ('로테이션 타이밍',  'rotation_timing_score',    '{:.3f}  (0=선점 1=후행)'),
        ('차량 이용률',      'vehicle_use_ratio',         '{:.1%}'),
        ('블루존 노출률',    'bluezone_exposure_ratio',   '{:.1%}'),
        ('안전구역 거리',    'safezone_proximity_mean',   '{:.0f}cm'),
        ('외곽 포지션 비율', 'safezone_edge_ratio',       '{:.3f}'),
        ('고도 변화량',      'altitude_variance',         '{:.1f}cm'),
    ]
    for label, col, fmt in metrics:
        val = p.get(col)
        if val is not None and pd.notna(val):
            print(f'  {label:<18}: {fmt.format(val)}')

    if 'win_flag' in p.index and pd.notna(p['win_flag']):
        print('-' * 55)
        print('  [이 판 결과]')
        print(f'  우승 여부: {"우승" if p["win_flag"] == 1 else "탈락"}')
        if 'winPlace'    in p.index: print(f'  최종 순위: {int(p["winPlace"])}위')
        if 'kills'       in p.index: print(f'  킬       : {p["kills"]:.0f}킬')
        if 'damageDealt' in p.index: print(f'  데미지   : {p["damageDealt"]:.0f}')

    peers = df_target[
        (df_target['persona'] == persona) & (df_target['accountId'] != account_id)
    ]
    if len(peers) > 0 and 'win_flag' in peers.columns:
        print('-' * 55)
        print(f'  [동일 페르소나 평균  n={len(peers):,}명]')
        print(f'  평균 우승률: {peers["win_flag"].mean():.1%}')
        if 'kills' in peers.columns:
            print(f'  평균 킬    : {peers["kills"].mean():.2f}')
    print('=' * 55)


# 샘플 accountId 출력
print('사용법: diagnose_player("account.xxxx", "Erangel")')
print()
if 'Erangel' in map_dfs:
    samples = map_dfs['Erangel']['accountId'].dropna().unique()[:5]
    print('샘플 accountId:')
    for s in samples:
        print(f'  {s}')
    print()
    if len(samples) > 0:
        diagnose_player(samples[0], 'Erangel')


In [ ]:
# 핵심 인사이트 요약
if len(df_all) > 0 and 'win_flag' in df_all.columns:
    print('=' * 60)
    print('  PUBG 전략 분석 - 핵심 인사이트 요약')
    print('=' * 60)

    best = df_all[df_all['cluster'] >= 0].groupby('persona')['win_flag'].mean()
    best_persona = best.idxmax()
    print(f'\n전체 맵 통합 최고 승률 전략:')
    print(f'  {best_persona}: {best.max():.1%}')

    print(f'\n맵별 최적 전략:')
    for map_name, df_m in map_dfs.items():
        valid = df_m[df_m['cluster'] >= 0]
        if len(valid) > 0 and 'win_flag' in valid.columns:
            wr = valid.groupby('persona')['win_flag'].mean()
            if len(wr) > 0:
                print(f'  {map_name:<12}: {wr.idxmax()} ({wr.max():.1%})')

    if 'damageDealt' in df_all.columns and 'safezone_proximity_mean' in df_all.columns:
        print(f'\n실력(데미지) vs 전략(포지셔닝) - 우승 상관관계:')
        corr_dmg = df_all['damageDealt'].corr(df_all['win_flag'])
        corr_pos = df_all['safezone_proximity_mean'].corr(df_all['win_flag'])
        print(f'  데미지    x 우승: {corr_dmg:.3f}')
        print(f'  포지셔닝  x 우승: {corr_pos:.3f}')
        if abs(corr_pos) > abs(corr_dmg):
            print('  -> 전략(포지셔닝)이 실력(데미지)보다 우승에 더 큰 영향')
        else:
            print('  -> 실력(데미지)이 전략(포지셔닝)보다 우승에 더 큰 영향')

    print()
    print('=' * 60)
    print(f'분석 완료!  결과 위치: {OUTPUT_DIR}')
    print('=' * 60)
